# Cycle through each year and extract the major topics

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
from sklearn.externals import joblib
from sklearn import decomposition


In [2]:
%matplotlib inline

In [3]:
total_bldgsim = pd.read_pickle("/Users/nus/twenty-years-of-bldgsim-textmining/total_email_data.pkl")

In [4]:
total_bldgsim.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20029 entries, 1999-03-04 08:26:46 to 2018-11-08 18:42:22
Data columns (total 4 columns):
From        20029 non-null object
DateTime    20029 non-null datetime64[ns]
Subject     20029 non-null object
Body        20028 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 782.4+ KB


# Loop through each year and calculate the tf-idf and the topics for each year

In [5]:
def rank_terms( A, terms ):
    # get the sums over each column
    sums = A.sum(axis=0)
    # map weights to the terms
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # rank the terms by their weight over all documents
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [12]:
def get_descriptor( terms, H, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( terms[term_index] )
    top_terms = [item for item in top_terms if not item.isdigit()]
    return top_terms

In [28]:
k = 6

# Run through each year, get the top 10 topics and their top 10 words

Put them in a dataframe with the year

In [29]:
all_topics = []
condensed_topics = []

In [30]:
for year in total_bldgsim.index.year.unique():
    bldg_sim_peryear = total_bldgsim[total_bldgsim.index.year == year]
    out_array = np.array(bldg_sim_peryear.fillna("x").Body)
    custom_stop_words = []
    with open( "/Users/nus/twenty-years-of-bldgsim-textmining/stopwords_annual_analysis_3.txt", "r" ) as fin:
        for line in fin.readlines():
            custom_stop_words.append( line.strip() )
    print(year)
    
    # use a custom stopwords list, set the minimum term-document frequency to 20
    vectorizer = CountVectorizer(stop_words = custom_stop_words, min_df = 20, encoding='latin-1')
    A = vectorizer.fit_transform(out_array)
    print( "Created %d X %d document-term matrix" % (A.shape[0], A.shape[1]) )
    
    # extract the resulting vocabulary
    terms = vectorizer.get_feature_names()
    print("Vocabulary has %d distinct terms" % len(terms))
    
    # we can pass in the same preprocessing parameters
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, min_df = 20, encoding='latin-1')
    A = vectorizer.fit_transform(out_array)
    print("Created %d X %d TF-IDF-normalized document-term matrix" % (A.shape[0], A.shape[1]) )
    
    joblib.dump((A,terms), "/Users/nus/twenty-years-of-bldgsim-textmining/emails-raw"+str(year)+".pkl") 
    
    (A,terms) = joblib.load( "/Users/nus/twenty-years-of-bldgsim-textmining/emails-raw"+str(year)+".pkl" )
    print( "Loaded %d X %d document-term matrix" % (A.shape[0], A.shape[1]) )
    
    model = decomposition.NMF( init="nndsvd", n_components=k ) 
    # apply the model and extract the two factor matrices
    W = model.fit_transform( A )
    H = model.components_
    
    descriptors = []
    condensed = []
    for topic_index in range(k):
        descriptors.append( get_descriptor( terms, H, topic_index, 5) )
        str_descriptor = ", ".join( descriptors[topic_index] )
        condensed.append(str_descriptor)
        print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )
    condensed_topics.append(condensed)
    
    descriptors_df = pd.DataFrame(descriptors).T
    descriptors_df['year'] = year
    all_topics.append(descriptors_df)

1999
Created 152 X 42 document-term matrix
Vocabulary has 42 distinct terms
Created 152 X 42 TF-IDF-normalized document-term matrix
Loaded 152 X 42 document-term matrix
Topic 01: doe, 1e, user, version, software
Topic 02: research, ashrae, information, looking, bill
Topic 03: jason, administrator, consultant, analytics
Topic 04: number, jun, bill, see, large
Topic 05: systems, software, better, source, available
Topic 06: thermal, program, hourly, interested, web
2000
Created 234 X 131 document-term matrix
Vocabulary has 131 distinct terms
Created 234 X 131 TF-IDF-normalized document-term matrix
Loaded 234 X 131 document-term matrix
Topic 01: doe, doe2, energyplus, powerdoe, program
Topic 02: weather, hourly, john, website, find
Topic 03: heat, water, air, temperature
Topic 04: software, development, effort, university, available
Topic 05: ashrae, research, systems, projects, engineer
Topic 06: public, private, open, source, software
2001
Created 211 X 77 document-term matrix
Vocabular

Topic 01: performance, experience, webinar, research, leed
Topic 02: openstudio, mc_cid, mc_eid, uniqid, psdconsulting
Topic 03: ibpsa, usa, conference, news
Topic 04: weather, whiteboxtechnologies, joe, climate, epw
Topic 05: energyplus, workshop, openstudio, designbuilder, training
Topic 06: jones, jim, water, thermal, intended
2017
Created 406 X 490 document-term matrix
Vocabulary has 490 distinct terms
Created 406 X 490 TF-IDF-normalized document-term matrix
Loaded 406 X 490 document-term matrix
Topic 01: cgi, 40gmail, successfully, tresidder, entitled
Topic 02: network, gototraining, attendee, performance, register
Topic 03: jim, buildingperformanceteam, dirkes, leed, linkedin
Topic 04: ibpsa, usa, buildingsimulation2017, news, conference
Topic 05: energyplus, openstudio, workshop, bigladdersoftware, training
Topic 06: weather, joe, whiteboxtechnologies, doe, solar
2018
Created 285 X 413 document-term matrix
Vocabulary has 413 distinct terms
Created 285 X 413 TF-IDF-normalized doc

In [31]:
condensed_topics = pd.DataFrame(condensed_topics)

In [32]:
condensed_topics.to_excel("condensed_topics.xlsx")

In [17]:
all_topics_df = pd.concat(all_topics).reset_index(drop=True)

In [18]:
all_topics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 9 columns):
0       255 non-null object
1       255 non-null object
2       250 non-null object
3       253 non-null object
4       255 non-null object
5       252 non-null object
6       246 non-null object
7       247 non-null object
year    260 non-null int64
dtypes: int64(1), object(8)
memory usage: 18.4+ KB


In [122]:
all_topics_df

,0,1,2,3,4,5,6,7,8,9,year
0,doe,ashrae,jason,number,systems,thermal,software,source,research,information,1999
1,1e,bill,administrator,jun,better,program,experience,hourly,project,interested,1999
2,user,0400,consultant,see,available,interested,looking,wrote,0700,looking,1999
3,version,large,analytics,large,source,web,0600,good,phone,site,1999
4,0400,interested,user,bill,text,tools,better,0700,good,web,1999
5,input,0500,wrote,better,large,project,tools,1e,jun,good,1999
6,text,jun,0600,hourly,web,phone,04,interested,program,bill,1999
7,experience,looking,input,04,tools,0500,web,references,experience,available,1999
8,0700,references,0500,phone,04,see,program,looking,available,tools,1999
9,project,text,text,doe,interested,version,version,program,analytics,phone,1999


In [123]:
all_topics_df_melted = all_topics_df.melt(id_vars='year')

In [124]:
all_topics_df_melted.head()

,year,variable,value
0,1999,0,doe
1,1999,0,1e
2,1999,0,user
3,1999,0,version
4,1999,0,0400


In [125]:
all_topics_df_melted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
year        2000 non-null int64
variable    2000 non-null object
value       2000 non-null object
dtypes: int64(1), object(2)
memory usage: 46.9+ KB


In [126]:
wordfreq = all_topics_df_melted.value.value_counts()

In [127]:
wordfreq.head(40)

leed           24
energyplus     20
cooling        20
ashrae         18
heat           18
weather        18
equest         17
performance    16
software       16
air            15
baseline       15
doe            15
modeling       15
experience     15
source         14
program        14
fan            14
ibpsa          14
available      13
water          13
engineering    13
information    12
web            12
systems        12
heating        12
usa            12
thermal        12
temperature    11
doe2           11
0500           11
training       10
conference     10
university     10
joe            10
load           10
yahoo           9
jeff            9
research        9
version         9
interested      8
Name: value, dtype: int64

In [128]:
wordfreq

leed                   24
energyplus             20
cooling                20
ashrae                 18
heat                   18
weather                18
equest                 17
performance            16
software               16
air                    15
baseline               15
doe                    15
modeling               15
experience             15
source                 14
program                14
fan                    14
ibpsa                  14
available              13
water                  13
engineering            13
information            12
web                    12
systems                12
heating                12
usa                    12
thermal                12
temperature            11
doe2                   11
0500                   11
                       ..
qm                      1
espr                    1
recommendations         1
dcmit_esiyok            1
qc                      1
lexington               1
bat                     1
eskewdumezri

In [129]:
wordfreq.to_csv("word_frequency.csv")